In [34]:
import socket
import struct
import time
from datetime import datetime, timedelta
import threading
import sys

In [35]:
class AbstractClock():
    def __init__(self):

        # Clock correction parameters
        self.delay = 0
        self.last_offset = 0
        self.offset = 0
        self.rate = 1
        self.last_rate = 1
        
        # NTP client
        self.ntp_client = NTPclient()
        
        # Time parameters for rate
        self.start_time = datetime.now()
        self.start_monotonic = time.monotonic()
        self.ntp_timestamp = ntp_client.getServerTime()[2]
        self.last_ntp_timestamp =  self.ntp_timestamp

        # intrinsic clock attributes
        self.timestamp = time.monotonic()
        self.last_timestamp = time.monotonic()
        self.correctedTimestamp = time.monotonic()


    def correctClock(self):
        ntp_time = self.ntp_client.getServerTime()
        t0 = datetime.strptime(ntp_time[1], '%Y-%m-%d %H:%M:%S.%f')
        t1 = datetime.strptime(ntp_time[2], '%Y-%m-%d %H:%M:%S.%f')
        t2 = datetime.strptime(ntp_time[3], '%Y-%m-%d %H:%M:%S.%f')
        t3 = datetime.strptime(ntp_time[4], '%Y-%m-%d %H:%M:%S.%f')

        # Update the rate parameters
        self.last_ntp_timestamp = self.ntp_timestamp
        self.ntp_timestamp = t1
        self.last_timestamp = self.timestamp
        self.timestamp = time.monotonic()



    def updateOffset(self):
        self.last_offset = self.offset
        self.offset = ((t1-t0) + (t2-t3)) / 2

    def updateRate(self):
        self.last_rate = self.rate
        self.rate = (self.timestamp - self.last_timestamp) / (self.ntp_timestamp - self.last_timestamp)

    def updateDelay(self):
        self.delay = (t3-t0) - (t2-t1)




In [36]:
class NTPclient():
    def __init__(self, update_rate):
        # Connection parameters
        self.host = '0.pool.ntp.org' # case of NTP online server
        #self.host = "10.42.0.1" # case of rasp
        #self.host = "192.168.0.0"
        self.port = 123
        self.read_buffer = 1024
        self.address = (self.host, self.port)
        self.epoch = 2208988800
        self.client = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

        # Fetch parameters
        self.update_rate = update_rate # seconds
        self.fetch_time = threading.Event()

    def start(self):
        while not self.fetch_time.wait(self.update_rate):
            global counter # DEBUG
            counter = 0 # DEBUG
            t = self.getServerTime()
            
    
    def getServerTime(self):
        data = b'\x1b' + 47 * b'\0'
        orig_time = datetime.now()
        self.client.sendto(data, self.address)
        print("Request sent")
        data, self.address = self.client.recvfrom(self.read_buffer)
        print("Response received")
        orig_int, orig_frac, ref_int, ref_frac, rx_int, rx_frac, tx_int, tx_frac = struct.unpack("!12I", data)[4:12] # 6:8 -> reference | 
        #ref_int -= self.epoch
        orig_int -= self.epoch
        rx_int -= self.epoch
        tx_int -= self.epoch

        # Convert the fractional part to microseconds (1 second = 2**32 fractional units)
        orig_frac = orig_frac * 1e6 // 2**32
        ref_frac = ref_frac * 1e6 // 2**32
        rx_frac = rx_frac * 1e6 // 2**32
        tx_frac = tx_frac * 1e6 // 2**32

        # Format the NTP time as a string
        ref_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(ref_int)), ref_frac)
        orig_time = orig_time.strftime('%Y-%m-%d %H:%M:%S.%f')
        rx_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(rx_int)), rx_frac)
        tx_time = "%s.%06d" % (time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(tx_int)), tx_frac)
        dest_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

        return ref_time, orig_time, rx_time, tx_time, dest_time

In [37]:
ABSTRACT_CLOCK_UPDATE_RATE = 0.1 # seconds 
NTP_UPDATE_RATE = 10 * ABSTRACT_CLOCK_UPDATE_RATE # seconds -> This can't be to low so that we have time to fetch the NTP time


ntp_client = NTPclient(NTP_UPDATE_RATE)


ntp_time = ntp_client.getServerTime()
print(ntp_time)

t0 = datetime.strptime(ntp_time[1], '%Y-%m-%d %H:%M:%S.%f')
t1 = datetime.strptime(ntp_time[2], '%Y-%m-%d %H:%M:%S.%f')
t2 = datetime.strptime(ntp_time[3], '%Y-%m-%d %H:%M:%S.%f')
t3 = datetime.strptime(ntp_time[4], '%Y-%m-%d %H:%M:%S.%f')


print("t0", t0)

offset = ((t1-t0) + (t2-t3)) / 2

print("Offset", offset)




Request sent
Response received


AttributeError: 'float' object has no attribute 'strftime'